In [4]:
from testcontainers.core.container import DockerContainer
from testcontainers.core.waiting_utils import wait_for_logs
from langchain_community.chat_models import ChatOllama
import docker

In [7]:
ollama = DockerContainer("ollama/ollama").with_kwargs(
    device_requests=[docker.types.DeviceRequest(count=-1, capabilities=[['gpu']])]).with_exposed_ports(11434)

ollama.start()
wait_for_logs(ollama, r".*Listening on.*")

Pulling image testcontainers/ryuk:0.7.0


Container started: 27dbfff1bec3
Waiting for container <Container: 27dbfff1bec3> with image testcontainers/ryuk:0.7.0 to be ready ...
Pulling image ollama/ollama
Container started: 5e9a86dd8937


1.0541212558746338

In [8]:
ollama.exec("ollama run llama3.1")

ExecResult(exit_code=0, output=b'\x1b[?25lpulling manifest \xe2\xa0\x8b \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\x99 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb8 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb8 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb4 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb4 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xa6 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xa7 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\x8f \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\x8b \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\x8b \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\x99 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb8 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb8 \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xbc \x1b[?25h\x1b[?25l\x1b[2K\x1b[1Gpulling manifest \xe2\xa0\xb

In [9]:
chat = ChatOllama(
            temperature=0,
            model="llama3.1",
            base_url=f'http://{ollama.get_container_host_ip()}:{ollama.get_exposed_port(11434)}',
            streaming=True,
            # seed=2,
            top_k=10,  # A higher value (100) will give more diverse answers, while a lower value (10) will be more conservative.
            top_p=0.3,  # Higher value (0.95) will lead to more diverse text, while a lower value (0.5) will generate more focused text.
            num_ctx=3072,  # Sets the size of the context window used to generate the next token.
        )

Waiting for container <Container: 5e9a86dd8937> with image ollama/ollama to be ready ...


In [10]:
message = chat.invoke("How are you?")
print(message.content)

I'm functioning well, thank you for asking. How can I assist you today?


In [12]:
from datasets import load_dataset

ds = load_dataset("PolyAI/banking77")

/home/chemmer/Repositories/docker-llm-demo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import pandas as pd
test_df = pd.DataFrame(ds["test"])

In [11]:
labels = ["activate_my_card", "age_limit", "apple_pay_or_google_pay", "atm_support", "automatic_top_up", "balance_not_updated_after_bank_transfer", "balance_not_updated_after_cheque_or_cash_deposit", "beneficiary_not_allowed", "cancel_transfer", "card_about_to_expire", "card_acceptance", "card_arrival", "card_delivery_estimate", "card_linking", "card_not_working", "card_payment_fee_charged", "card_payment_not_recognised", "card_payment_wrong_exchange_rate", "card_swallowed", "cash_withdrawal_charge", "cash_withdrawal_not_recognised", "change_pin", "compromised_card", "contactless_not_working", "country_support", "declined_card_payment", "declined_cash_withdrawal", "declined_transfer", "direct_debit_payment_not_recognised", "disposable_card_limits", "edit_personal_details", "exchange_charge", "exchange_rate", "exchange_via_app", "extra_charge_on_statement", "failed_transfer", "fiat_currency_support", "get_disposable_virtual_card", "get_physical_card", "getting_spare_card", "getting_virtual_card", "lost_or_stolen_card", "lost_or_stolen_phone", "order_physical_card", "passcode_forgotten", "pending_card_payment", "pending_cash_withdrawal", "pending_top_up", "pending_transfer", "pin_blocked", "receiving_money", "Refund_not_showing_up", "request_refund", "reverted_card_payment?", "supported_cards_and_currencies", "terminate_account", "top_up_by_bank_transfer_charge", "top_up_by_card_charge", "top_up_by_cash_or_cheque", "top_up_failed", "top_up_limits", "top_up_reverted", "topping_up_by_card", "transaction_charged_twice", "transfer_fee_charged", "transfer_into_account", "transfer_not_received_by_recipient", "transfer_timing", "unable_to_verify_identity", "verify_my_identity", "verify_source_of_funds", "verify_top_up", "virtual_card_not_working", "visa_or_mastercard", "why_verify_identity", "wrong_amount_of_cash_received", "wrong_exchange_rate_for_cash_withdrawal"]

#### testing

In [19]:
samples = []
for idx, row in df.sample(10).iterrows():
    samples.append((row["text"],row["label"]))

In [23]:
df.shape

(10003, 2)

In [21]:
for sample in samples:
    prompt = f"""
    Take the following customer text and classify it into one of the following classes. Answer only with the corresponding class
    classes:
    {labels}
    text:
    {sample[0]}
    """
    print(prompt)


    Take the following customer text and classify it into one of the following classes. Answer only with the corresponding class
    classes:
    ['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_state

In [ ]:
transfer_into_account
compromised_card
pending_card_payment
direct_debit_payment_not_recognised
atm_support

In [34]:
for sample in samples:
    prompt = f"""
    Take the following customer text and classify it into one of the following classes. Answer only with the corresponding class
    classes:
    {labels}
    text:
    {sample[0]}
    """
    message = chat.invoke(prompt)
    print(message.content)

transfer_into_account
'compromised_card'
'pending_card_payment'
'direct_debit_payment_not_recognised'
'cash_withdrawal_charge'
'activate_my_card'
'find_missing_payment_details'
exchange_rate
'change_pin'
request_refund


#### zero-shot

In [15]:
def invoke_llama(text):
    prompt = f"""
    Take the following customer text and classify it into one of the following classes. Answer only with the corresponding class
    classes:
    {labels}
    text:
    {text}
    """
    message = chat.invoke(prompt)
    return message.content

test_df["pred"] = test_df["text"].apply(invoke_llama)

In [35]:
def get_class_description(label, examples):
    prompt = f"""
    Take the classification label for customer texts and examples for it and write a description of the class to help you classify it in the future. Answer with only the description.
    class name:
    {label}
    example texts:
    {examples}
    """
    message = chat.invoke(prompt)
    return message.content
label_descriptions = {}

for label in labels:
    examples = "\n".join(df[df["label_name"]==label].sample(5)["text"].to_list())
    description = get_class_description(label,examples)
    print(description)
    label_descriptions[label] = description

Customer is experiencing issues with activating their card.
Age-related inquiries or concerns about customer eligibility.
"Questions or issues related to topping up funds using Apple Pay or Google Pay."
Customer inquiries about accessing or using their Mastercard at an ATM.
Automatic bank account topping up, allowing for scheduled or recurring payments to maintain a minimum balance.
Customer is experiencing issues with their bank account, specifically that their balance has not been updated after a bank transfer.
Customer has made a payment (cheque or cash deposit) but their account balance is not reflecting the update.
Customer is unable to complete a transaction, specifically a transfer of funds or cryptocurrency, due to restrictions or limitations.
Request for urgent assistance to cancel or reverse a mistaken payment transfer due to incorrect account information, often requiring immediate action to rectify the situation.
Customer inquiries about expiring bank cards and the process o

In [48]:
labels_with_description = []
for label in labels:
    labels_with_description.append(label + " description: "  +label_descriptions[label])

def invoke_llama_with_description(text):
    prompt = f"""
    Take the following customer text and classify it into one of the following classes. Answer only with the corresponding class
    classes:
    {labels_with_description}
    text:
    {text}
    """
    message = chat.invoke(prompt)
    return message.content

test_df["pred"] = test_df["text"].apply(invoke_llama_with_description)

In [17]:
test_df.to_parquet("banking_test_llama31.parquet")

In [2]:
import pandas as pd
#df.to_parquet("banking_train_llama31.parquet")

df = pd.read_parquet("banking_train_llama31.parquet")

In [34]:
def calculate_label_name(label):
    return labels[label]
test_df["label_name"] = test_df["label"].apply(calculate_label_name)
df["label_name"] = df["label"].apply(calculate_label_name)

In [24]:
def calculate_label(pred):
    pred = pred.replace("'","")
    if pred in labels:
        return labels.index(pred)
    return -1

test_df["pred_label"] = test_df["pred"].apply(calculate_label)

In [26]:
test_df[test_df["pred_label"]==-1]

,text,label,pred,pred_label
53,How do I add a card to the app?,13,'add_card',-1
160,My statement has a dollar I have been charged ...,34,'refund_not_showing_up',-1
169,"I do not remember purchasing anything for 1£, ...",34,'unknown_transaction_on_statement',-1
191,There is a transaction for $1 and I don't know...,34,'unknown_transaction_charge',-1
229,I have a pending transaction in my app. How is...,46,pending_transaction_status,-1
...,...,...,...,...
2992,How can I top-ff my account using my Apple Watch?,2,top_up_by_app,-1
2996,How do I top up with Apple Pay?,2,top_up_by_apple_pay,-1
2998,How do I use the top up app with my apple watch?,2,'get_virtual_card',-1
2999,"Google pay and top up, I want it, can I get it?",2,'get_virtual_card',-1


In [27]:
test_df["is_correct"] = test_df["pred_label"] == test_df["label"]

In [31]:
test_df.is_correct.value_counts()

is_correct
True     1675
False    1405
Name: count, dtype: int64

Results

Zero shot: 54,4 % Accuracy


In [32]:
test_df[test_df["is_correct"]== False]

,text,label,pred,pred_label,is_correct,label_name
1,"I still have not received my new card, I order...",11,'getting_spare_card',39,False,card_arrival
8,still waiting on that card,11,'getting_spare_card',39,False,card_arrival
10,How do I track my card?,11,'activate_my_card',0,False,card_arrival
11,How long does a card delivery take?,11,'card_delivery_estimate',12,False,card_arrival
12,I still don't have my card after 2 weeks. Wha...,11,'getting_spare_card',39,False,card_arrival
...,...,...,...,...,...,...
3054,"Please help me get a new card, I reside in the...",24,'get_physical_card',38,False,country_support
3058,Where can I find your locations?,24,'unknown_class',-1,False,country_support
3066,"I need a card, but I'm in the US at the moment.",24,'getting_virtual_card',40,False,country_support
3074,"I'm not in the UK, can I get a card?",24,'get_physical_card',38,False,country_support
